# Generating the synthetic stations and their apparatus assignments

In [ ]:
import pandas as pd
assignment_cols = ['StationID', 'Engine', 'Truck', 'Rescue', 'Hazard', 'Squad', 'FAST', 'Medic', 'Brush', 'Boat', 'UTV', 'REACH']
station_cols = ['StationID', 'Facility Name', 'Address', 'City', 'State', 'Zip Code', 'GLOBALID', 'lat', 'lon']
sdf = pd.read_csv("/Users/jose/Developer/git/fire_simulator/data/stations.csv")
sdf.loc[sdf['Facility Name'] == 'Station 41 / Goodlettesville', 'Facility Name'] = 'Station 41'
sdf['StationID'] = sdf['Facility Name'].str.split(' ').str[1]
sdf = sdf[station_cols]
sdf['StationID'] = sdf['StationID'].astype('int')
sdf = sdf.sort_values(by='StationID')
sdf = sdf.reset_index(drop=True)
sdf['StationID'] = sdf.index
sdf.head()
assignments = []
for k, v in sdf.iterrows():
    values = [v['StationID']] + [2] + [0] * (len(assignment_cols) - 2)
    assignments.append(values)
adf = pd.DataFrame(assignments, columns=assignment_cols)
adf.to_csv("./data/assignments.csv", index=False)
sdf.to_csv("./data/stations.csv", index=False)

In [ ]:
import geopandas as gpd
# Get actual bounds of the city.
df = gpd.read_file("data/bounds.geojson")
df.plot()
df.minimum_rotated_rectangle().to_file("data/bounds_mrr.geojson", driver="GeoJSON")
df.plot()

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
stations = pd.read_csv("data/stations.csv")
incidents = pd.read_csv("data/incidents.csv")
# plot the two dataframes on a map
stations_gdf = gpd.GeoDataFrame(
    stations, geometry=gpd.points_from_xy(stations.lon, stations.lat), crs="EPSG:4326"
)
incidents_gdf = gpd.GeoDataFrame(
    incidents, geometry=gpd.points_from_xy(incidents.lon, incidents.lat), crs="EPSG:4326"
)
fig, ax = plt.subplots(figsize=(10, 10))
stations_gdf.plot(ax=ax, color='blue', markersize=5, label='Stations')
incidents_gdf.plot(ax=ax, color='red', markersize=5, label='Incidents')
# add the station and incident labels
for x, y, label in zip(stations_gdf.geometry.x, stations_gdf.geometry.y, stations_gdf['StationID']):
    ax.text(x, y, label, fontsize=8, ha='right', va='bottom', color='blue')
for x, y, label in zip(incidents_gdf.geometry.x, incidents_gdf.geometry.y, incidents_gdf['incident_id']):
    ax.text(x, y, label, fontsize=8, ha='right', va='bottom', color='red')
plt.legend()
plt.title('Fire Stations and Incidents')

In [ ]:
import pandas as pd
date_range = pd.date_range(start="2022-01-01", end="2022-12-31", freq='1D')
# 325002//len(date_range), len(date_range)
PER_DAY = 314
COUNT = PER_DAY * len(date_range)
COUNT

# Generating synthetic incident data

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import random
import requests
from pandas import to_datetime, Timedelta

def generate_random_points_within_bounds(bounds, num_points):
    minx, miny, maxx, maxy = bounds.bounds
    points = []
    for _ in range(num_points):
        epsilon = 1e-8  # A very small value
        x = random.uniform(minx + epsilon, maxx - epsilon)
        y = random.uniform(miny + epsilon, maxy - epsilon)
        points.append(Point(x, y))
    return points

def get_nearest_lat_lon(point, osrm_url):
    response = requests.get(f"{osrm_url}/nearest/v1/driving/{point.x},{point.y}")
    if response.status_code == 200:
        data = response.json()
        if data['waypoints']:
            return data['waypoints'][0]['location']
    return None

def generate_incidents(num_incidents, incident_types, incident_levels, date_range, nearest_points):
    incidents = []
    for idx in range(num_incidents):
        incident_type = random.choice(incident_types)
        incident_level = random.choice(incident_levels)
        date_time = random.choice(date_range)
        lon, lat = random.choice(nearest_points)
        incidents.append({
            "incident_id": f"{idx}",
            "lat": lat,
            "lon": lon,
            "incident_type": incident_type,
            "incident_level": incident_level,
            "datetime": date_time,
        })
    return pd.DataFrame(incidents)

osrm_url = "http://localhost:8085"
num_incidents = 400
nearest_points = []

# Get actual bounds of the city.
df = gpd.read_file("data/bounds.geojson")
# Generate random points within the bounds.
random_points = generate_random_points_within_bounds(df.unary_union, num_incidents)
# for each point, query the nearest lat lon on the street network with osrm
for point in random_points:
    nearest = get_nearest_lat_lon(point, osrm_url)
    if nearest:
        nearest_points.append(nearest)

incident_types = ["Fire"]
incident_levels = ["Low", "Moderate", "High", "Critical"]
# date_range = pd.date_range(start="2022-01-01", end="2024-10-29", freq='1T')
date_range = pd.date_range(start="2022-01-01", end="2022-01-02", freq='5T')

incidents_df = generate_incidents(num_incidents, incident_types, incident_levels, date_range, nearest_points)
incidents_df = incidents_df.sort_values(by='datetime')
incidents_df = incidents_df.reset_index(drop=True)
incidents_df['incident_id'] = incidents_df.index


# Ensure datetime is in datetime format and sorted

seen = set()
for idx, row in incidents_df.iterrows():
    dt = row['datetime']
    # Increment by 1 minute until unique
    while dt in seen:
        dt += Timedelta(minutes=1)
    incidents_df.at[idx, 'datetime'] = dt
    seen.add(dt)

incidents_df['datetime'] = pd.to_datetime(incidents_df['datetime'])
incidents_df = incidents_df.sort_values('datetime').reset_index(drop=True)
incidents_df.to_csv("data/incidents.csv", index=False)

# Plotting the differences between different OSRM instances
* One is turning off wrong way driving

In [ ]:
# Given this url: http://localhost:8080/route/v1/driving/-86.7816,36.1627;-86.7679,36.1745?overview=full&geometries=geojson
# plot the route on the map.
import requests
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import shape

def plot_route(osrm_url, start, end):
    response = requests.get(f"{osrm_url}/route/v1/driving/{start[0]},{start[1]};{end[0]},{end[1]}?overview=full&geometries=geojson")
    if response.status_code == 200:
        data = response.json()
        if data['routes']:
            route_geojson = data['routes'][0]['geometry']
            route_shape = shape(route_geojson)
            route_gdf = gpd.GeoDataFrame(geometry=[route_shape], crs="EPSG:4326")
            return route_gdf
    return None

# Normal car version
osrm_url = "http://localhost:8080"

start = (-86.7816, 36.1627)
end = (-86.7679, 36.1745)

route_gdf = plot_route(osrm_url, start, end)

fig, ax = plt.subplots()
if route_gdf is not None:
    route_gdf.plot(ax=ax, color='blue', linewidth=5, label='CAR')

# EMS version
osrm_url = "http://localhost:8085"

start = (-86.7816, 36.1627)
end = (-86.7679, 36.1745)

route_gdf = plot_route(osrm_url, start, end)

if route_gdf is not None:
    route_gdf.plot(ax=ax, color='red', linewidth=2, label='EMS')
    ax.set_title("Route from Start to End")

ctx.add_basemap(ax, crs=route_gdf.crs.to_string())
plt.legend()
plt.show()


# Cleaning and Preparing the Zones geojson
* Split multipolygons into separate polygons, but keep the names and zones.

In [13]:
import geopandas as gpd
import os
import pandas as pd
from shapely.geometry import Point
import re

def split_string(s):
    match = re.match(r'^(\d+)(.*)$', s)
    if match:
        number = int(match.group(1))
        suffix = match.group(2) if match.group(2) else None
        return number, suffix
    else:
        raise ValueError(f"Invalid input: {s}")

# Suppose 'union_polygon' may be a MultiPolygon or Polygon
def split_to_polygons(geom):
    if geom.geom_type == 'MultiPolygon':
        return list(geom.geoms)
    elif geom.geom_type == 'Polygon':
        return [geom]
    else:
        return []
    
df = gpd.read_file("data/FireBeats_shapefile_05152025")
# Convert to 4326
df = df.to_crs(epsg=4326)
df_arr = []
for k, v in df.groupby("NAME"):
    union_polygon = v.union_all()
    # split the name into int and string components

    i_name, s_name = split_string(k)
    if s_name is not None:
        name = f"{str(i_name).zfill(2)}{s_name}"
    else:
        name = str(i_name).zfill(2)
    polygons = split_to_polygons(union_polygon)
    for poly in polygons:
        data = {
            "NAME": name,
            "ZONE": v["ZONE"].iloc[0],
            "TYPE": v["TYPE"].iloc[0],
            "geometry": poly
        }
        df_arr.append(data)
df = gpd.GeoDataFrame(df_arr, crs="EPSG:4326")
df.to_file("data/beats_shpfile.geojson", driver="GeoJSON")

# Setting up the beats matrix
* Requires the FIRE RUN CARDS OCT 2024 folder to be present.
* Fire run cards directory should have files named:
    * Beat-ZONE.xlsx

In [4]:
import geopandas as gpd
import pandas as pd
import os

In [5]:
df = gpd.read_file("data/FireBeats_shapefile_05152025")
zones = df['NAME'].unique()
zones = sorted(zones)
zones[1:5]

# What are these names and what do they correspond to?
ignored_names = [
    "FD/RADIO",
    "FD/FST11",
    "FD/FST06",
    "FD/FST09",
    "FD/FST37",
    "FD/FAST01",
    "FD/SQ01",
    "FD/SQ37",
    "FD/SQ09",
    "FD/SQ11",
    "FD/TC05",
    "FD/DS31",
    "FD/MED41",
    "FD/HQ",
    "FD/BAR*",
    "FD/DSOP",
    "FD/BT13", "FD/BT22", "FD/BT35", "FD/BT36"
]

In [24]:


# Read the Excel file and specify the sheet name (or use sheet number)
excel_path = "data/FIRE RUN CARDS OCT 2024/"
sheet_name = "RunCardOrder"  # Change to your actual sheet name

beats = []
max_runs = 0
for zone_name in zones:

    i_name, s_name = split_string(zone_name)
    # print(i_name, s_name)
    if s_name is not None:
        new_name = f"{str(i_name).zfill(2)}{s_name}"
    else:
        new_name = str(i_name).zfill(2)
    # Construct the full path to the Excel file for the specific zone
    excel_path = f"data/FIRE RUN CARDS OCT 2024/BEAT-{new_name}.xlsx"
    
    # Check if the file exists before proceeding
    if not os.path.exists(excel_path):
        print(f"File {excel_path} does not exist. Skipping.")
        continue
    
    # Read the sheet into a DataFrame
    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    # Confirm these with Fire Department
    df = df[~df['OrderValue'].isin(ignored_names)]
    df = df[~df['OrderValue'].str.contains("FST")]
    df = df[~df['OrderValue'].str.contains("FB")]
    df = df[~df['OrderValue'].str.contains("SQ")]
    df = df[~df['OrderValue'].str.contains("MED")]
    df = df[~df['OrderValue'].str.contains("BT")]
    df = df[~df['OrderValue'].str.contains("DSOP")]
    df = df[~df['OrderValue'].str.contains("RADIO")]
    df = df[~df['OrderValue'].str.contains("HQ")]
    df = df[~df['OrderValue'].str.contains("EN")]
    df = df[~df['OrderValue'].str.contains("RE")]
    df = df[~df['OrderValue'].str.contains("ATV")]
    df = df[~df['OrderValue'].str.contains("TC")]
    df = df[~df['OrderValue'].str.contains("BAR")]
    df['Facility Name'] = df['OrderValue'].str.split('/').str[1]
    try:
        df['Facility Name'] = df['Facility Name'].str.replace('S', 'Station ')
        df['StationID'] = df['Facility Name'].str.split(' ').str[1]
        df['StationID'] = df['StationID'].astype('int')
        df['Facility Name'] = 'Station ' + df['StationID'].astype(str).str.zfill(2)
        df.drop(columns=['StationID'], inplace=True)
    except Exception as e:
        print(new_name, e)
        # display(df.head())

    max_runs = max(max_runs, df['OrderValue'].shape[0])
    run_order = df['Facility Name'].tolist()
    if len(run_order) < max_runs:
        run_order += ["None"] * (max_runs - len(run_order))
    run_order = [new_name] + run_order
    beats.append(run_order)
df = pd.DataFrame(beats, columns=['Zone'] + [f'Run {i+1}' for i in range(max_runs)])
# replace all '' in df with 'None'
df = df.replace('', 'None')
df = df.fillna('None')
for i in range(len(beats)):
    # print(beats[i])
    if len(beats[i]) < max_runs:
        beats[i] += ["None"] * (max_runs - len(beats[i]))
    # print(len(beats[i]))
df.to_csv("data/beats.csv", index=False)

File data/FIRE RUN CARDS OCT 2024/BEAT-10.xlsx does not exist. Skipping.
File data/FIRE RUN CARDS OCT 2024/BEAT-15A.xlsx does not exist. Skipping.
File data/FIRE RUN CARDS OCT 2024/BEAT-07R.xlsx does not exist. Skipping.


In [25]:
import struct

def save_string_matrix(filename, matrix):
    with open(filename, "wb") as f:
        height = len(matrix)
        width = len(matrix[0]) if height > 0 else 0
        f.write(struct.pack("ii", width, height))

        for row in matrix:
            for s in row:
                s_bytes = s.encode("utf-8")
                f.write(struct.pack("i", len(s_bytes)))
                f.write(s_bytes)
matrix = df.to_numpy().tolist()
save_string_matrix("logs/beats.bin", matrix)

In [8]:
len(beats[0])

41

In [9]:
df.Zone.unique()

array(['01', '10A', '10B', '10C', '10D', '10E', '10F', '11', '11B', '11C',
       '11D', '11E', '11F', '11G', '11H', '11R1', '11R2', '12', '12B',
       '12C', '12D', '12E', '12F', '12G', '12H', '12R', '13', '13B',
       '13C', '13D', '13E', '13F', '13G', '13H', '13I', '13J', '13K',
       '13R1', '13R2', '14', '14B', '14C', '14D', '14E', '14R1', '14R2',
       '15', '15B', '15C', '15D', '15E', '15F', '16', '16B', '16C', '16D',
       '16E', '16F', '16G', '16H', '17', '17A', '17B', '17C', '17D',
       '17E', '17F', '18', '18B', '18C', '18D', '18E', '18F', '18G',
       '18H', '18I', '18R', '19', '19B', '19C', '19D', '19E', '19F',
       '19G', '19H', '19I', '19J', '02', '20', '20B', '20C', '20D', '20E',
       '20F', '20G', '20H', '20K', '20L', '20M', '21', '21B', '21C',
       '21D', '21E', '21F', '21G', '21I', '21J', '21K', '22', '22B',
       '22C', '22D', '22R1', '22R2', '23', '23B', '23C', '23R1', '23R2',
       '24', '24B', '24C', '24D', '24E', '24F', '24R1', '24R2', '24R3',
  

In [10]:
zones

['1',
 '10',
 '10A',
 '10B',
 '10C',
 '10D',
 '10E',
 '10F',
 '11',
 '11B',
 '11C',
 '11D',
 '11E',
 '11F',
 '11G',
 '11H',
 '11R1',
 '11R2',
 '12',
 '12B',
 '12C',
 '12D',
 '12E',
 '12F',
 '12G',
 '12H',
 '12R',
 '13',
 '13B',
 '13C',
 '13D',
 '13E',
 '13F',
 '13G',
 '13H',
 '13I',
 '13J',
 '13K',
 '13R1',
 '13R2',
 '14',
 '14B',
 '14C',
 '14D',
 '14E',
 '14R1',
 '14R2',
 '15',
 '15A',
 '15B',
 '15C',
 '15D',
 '15E',
 '15F',
 '16',
 '16B',
 '16C',
 '16D',
 '16E',
 '16F',
 '16G',
 '16H',
 '17',
 '17A',
 '17B',
 '17C',
 '17D',
 '17E',
 '17F',
 '18',
 '18B',
 '18C',
 '18D',
 '18E',
 '18F',
 '18G',
 '18H',
 '18I',
 '18R',
 '19',
 '19B',
 '19C',
 '19D',
 '19E',
 '19F',
 '19G',
 '19H',
 '19I',
 '19J',
 '1B',
 '1C',
 '1D',
 '1E',
 '1F',
 '1G',
 '1H',
 '1I',
 '2',
 '20',
 '20B',
 '20C',
 '20D',
 '20E',
 '20F',
 '20G',
 '20H',
 '20K',
 '20L',
 '20M',
 '21',
 '21B',
 '21C',
 '21D',
 '21E',
 '21F',
 '21G',
 '21I',
 '21J',
 '21K',
 '22',
 '22B',
 '22C',
 '22D',
 '22R1',
 '22R2',
 '23',
 '23B',
 '